In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_pickle('CRSP.pickle')
data.columns

Index(['PERMNO', 'date', 'SHRCD', 'SICCD', 'TICKER', 'SHRCLS', 'NAICS',
       'PRIMEXCH', 'CUSIP', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET',
       'BID', 'ASK', 'SHROUT', 'OPENPRC', 'NUMTRD', 'RETX', 'vwretd', 'vwretx',
       'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')

# CHMOM: Cumulative return from t-6 to t-1 minus t-12 to t-7
Gettleman & Marks 2006, WP

In [3]:
def chmom():
    # Input Data
    df = data[['date', 'PERMNO', 'RET']].copy()

    df['date'] = pd.to_datetime(df['date'])
    df['year_month'] = df['date'].dt.to_period('M')

    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df.dropna(subset=['RET'], inplace=True)

    df = df.sort_values(by=['PERMNO', 'year_month'])
    
    # 使用log return加總，計算每個月報酬
    df['log_ret'] = np.log(df['RET']+1)

    month_ret = df.groupby(['PERMNO', 'year_month'])['log_ret'].sum().rename('month_log_ret').reset_index()
    
    #Rolling 6 month 計算累積六過月報酬率
    month_ret['rolling_6m_ret'] = month_ret.groupby('PERMNO')['month_log_ret']\
                                           .rolling(window=6).sum().reset_index(level=0, drop=True)
    
    # 將報酬率平移至對應的月份、還原為累積報酬率
    month_ret['t-6~t-1'] = np.exp(month_ret.groupby('PERMNO')['rolling_6m_ret'].shift(1))-1
    month_ret['t-12~t-7'] = np.exp(month_ret.groupby('PERMNO')['rolling_6m_ret'].shift(7))-1
    
    month_ret['chmom'] = month_ret['t-6~t-1']-month_ret['t-12~t-7']
    
    return month_ret[['PERMNO', 'year_month', 'chmom']].dropna().reset_index(drop = True)

chmom = chmom()
chmom.to_pickle('chmom.pickle')

# MAXRET: 前月單日最大報酬率
Bali, Cakici & Whitelaw, 2011. JFE

In [4]:
def maxret():
    # Input Data
    df = data[['date', 'PERMNO', 'RET']].copy()

    df['date'] = pd.to_datetime(df['date'])
    df['year_month'] = df['date'].dt.to_period('M')

    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df.dropna(subset=['RET'], inplace=True)

    df = df.sort_values(by=['PERMNO', 'year_month'])
    
    # 前月單日最大報酬率
    maxret = pd.DataFrame(df.groupby(['PERMNO', 'year_month'])['RET'].agg('max').rename('maxret')).reset_index()
    maxret['year_month']+=1
    
    return maxret.dropna()

maxret = maxret()
maxret.to_pickle('maxret.pickle')

# MOM12M: 12個月累積報酬
Jagadeesh, 1990, JF

# MOM1M: 前月累積報酬
Jagadeesh, Titman, 1993, JF

In [5]:
def mom():
    df = data[['date', 'PERMNO', 'RET']].copy()

    df['date'] = pd.to_datetime(df['date'])
    df['year_month'] = df['date'].dt.to_period('M')

    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df.dropna(subset=['RET'], inplace=True)

    df = df.sort_values(by=['PERMNO', 'year_month'])
    df['log_ret'] = np.log(df['RET']+1)

    month_ret = df.groupby(['PERMNO', 'year_month'])['log_ret'].sum().rename('month_log_ret').reset_index()
    month_ret['rolling_12m_ret'] = month_ret.groupby('PERMNO')['month_log_ret'].rolling(window=12).sum().reset_index(level=0, drop=True)

    month_ret['mom1m'] = np.exp(month_ret.groupby('PERMNO')['month_log_ret'].shift(1))-1
    month_ret['mom12m'] = np.exp(month_ret.groupby('PERMNO')['rolling_12m_ret'].shift(1))-1

    return month_ret[['PERMNO', 'year_month', 'mom1m', 'mom12m']].dropna().reset_index(drop = True)

mom = mom()
mom.to_pickle('momentum.pickle')

# Idiovol: 
**regress daily returns on a value-weighted market index over a one-year period immediately preceding the holding period and compute the variance of the residual term.**

Ali, Hwang & Trombley, 2003, JFE

In [7]:
# 1. 輸入資料、整理

df = data[['date', 'PERMNO', 'RET', 'vwretd']].copy()

df['date'] = pd.to_datetime(df['date'])
df['year_month'] = df['date'].dt.to_period('M')

df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
df['vwretd'] = pd.to_numeric(df['vwretd'], errors='coerce')

df.dropna(subset=['RET', 'vwretd'], inplace=True)

df = df.sort_values(by=['PERMNO', 'year_month'])

# Regression 取得 MSE
# 此處使用平行運算加速
import statsmodels.api as sm

def calculate_mse(dfc, year_month):
    end_date = year_month.to_timestamp()
    start_date = end_date - pd.DateOffset(months=12)
    
    try:
        data = dfc.loc[(dfc['date'] >= start_date) & (dfc['date'] < end_date)]
        X = data['vwretd']
        Y = data['RET']
        X = sm.add_constant(X)
        model = sm.OLS(Y, X).fit()
        mse = model.mse_resid
        return mse
    
    except Exception:
        return np.nan

def process_task(dfc):
    result = []
    for PERMNO in dfc['PERMNO'].unique():
        df_PERMNO = dfc[dfc['PERMNO'] == PERMNO]
        for year_month in df_PERMNO['year_month'].unique():
            mse = calculate_mse(df_PERMNO, year_month)
            result.append({'year_month': year_month, 'PERMNO': PERMNO, 'MSE': mse})
    return result

def create_tasks(df):
    unique_PERMNOs = df['PERMNO'].unique()
    PERMNO_chunks = [unique_PERMNOs[i:i + 2013] for i in range(0, len(unique_PERMNOs), 2013)]
    tasks = [df[df['PERMNO'].isin(chunk)] for chunk in PERMNO_chunks]
    return tasks

from pathos.multiprocessing import ProcessingPool as Pool

def main(df):
    tasks = create_tasks(df)
    pool = Pool(processes=8)  # Adjust number of processes based on your machine
    results = pool.map(process_task, tasks)
    pool.close()
    pool.join()
    return results

def combine_results(results):
    flat_results = [item for sublist in results for item in sublist]
    result_df = pd.DataFrame(flat_results)
    return result_df


all_results = main(df)
result_df = combine_results(all_results)

#整理計算結果、儲存
result_df_modified = result_df.groupby('PERMNO').apply(lambda x: x.iloc[12:] if len(x) > 12 else pd.DataFrame()).reset_index(drop=True)
result_df_modified.rename(columns = {'MSE':'Idiovol'}, inplace = True)
result_df_modified.to_pickle('idiovol.pickle')

# dolvol: 兩個月交易量（$ not #），取自然對數
Chrodia, Subrahmanyam & Anshuman, 2001, JFE

In [8]:
def dolvol():
    df = data[['date', 'PERMNO', 'VOL', 'PRC']].copy()

    df['date'] = pd.to_datetime(df['date'])
    df['year_month'] = df['date'].dt.to_period('M')

    df['VOL'] = pd.to_numeric(df['VOL'], errors='coerce')
    df['PRC'] = pd.to_numeric(np.abs(df['PRC']), errors='coerce')

    df.dropna(subset=['VOL', 'PRC'], inplace=True)
    
    df = df.sort_values(by=['PERMNO', 'year_month'])
    df['$VOL'] = df['VOL']*df['PRC']
    
    month_vol = df.groupby(['PERMNO', 'year_month'])['$VOL'].sum().rename('month_trading_volume').reset_index()
    month_vol['rolling_2m_volume'] = month_vol.groupby('PERMNO')['month_trading_volume'].rolling(window=2).sum().reset_index(level=0, drop=True)
    month_vol['year_month']+=1
    
    return month_vol[['PERMNO', 'year_month', 'dolvol']].dropna().reset_index(drop = True)

dolvol = dolvol()
dolvol.to_pickle('dolvol.pickle')

# retvol: Total return volatility

In [9]:
def retvol():
    df = data[['date', 'PERMNO', 'RET']].copy()

    df['date'] = pd.to_datetime(df['date'])
    df['year_month'] = df['date'].dt.to_period('M')

    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df.dropna(subset=['RET'], inplace=True)

    df = df.sort_values(by=['PERMNO', 'year_month'])

    month_vol = pd.DataFrame(df.groupby(['PERMNO', 'year_month'])['RET'].agg('std').rename('month_vol')).reset_index()
    month_vol['year_month']+=1
    
    return month_vol[['PERMNO', 'year_month', 'retvol']].dropna().reset_index(drop = True)

retvol = retvol()
retvol.to_pickle('retvol.pickle')

# indmom: industry momentum
Moskowitz & Grinblatt, 1999, JF

In [10]:
def indmom():
    # 1. Input Data
    df = data[['date', 'PERMNO', 'SICCD', 'RET', 'PRC', 'SHROUT']].copy()

    df['date'] = pd.to_datetime(df['date'])
    df['year_month'] = df['date'].dt.to_period('M')

    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df['PRC'] = pd.to_numeric(np.abs(df['PRC']), errors='coerce')
    df['SHROUT'] = pd.to_numeric(np.abs(df['SHROUT']), errors='coerce')
    df['SICCD'] = pd.to_numeric(df['SICCD'], errors='coerce')
    df.dropna(subset=['RET', 'PRC', 'SHROUT', 'SICCD'], inplace=True)

    df = df.sort_values(by=['PERMNO', 'year_month'])
    
    # 2. 計算個股每月報酬
    df['log_ret'] = np.log(df['RET']+1)
    df['stock_month_ret'] = np.exp(df.groupby(['PERMNO', 'year_month'])['log_ret'].transform('sum'))-1
    
    # 3. 計算個股市值
    df['market_value'] = df['PRC']*df['SHROUT']
    df['monthly_market_value'] = df.groupby(['PERMNO', 'year_month'])['market_value'].transform('mean')
    
    df = df[['year_month', 'PERMNO', 'SICCD', 'stock_month_ret', 'monthly_market_value']]
    df = df.drop_duplicates(subset = ['year_month', 'PERMNO'])
    
    # 4. 用 SICCD 開頭兩碼進行產業分類
    industry_mapping = {
    range(10, 15): 'Mining', #10~14
    range(20, 21): 'Food', #20
    range(22, 24): 'Apparel', #22~23
    range(26, 27): 'Paper', #26
    range(28, 29): 'Chemical', #28
    range(29, 30): 'Petroleum', #29
    range(32, 33): 'Construction', #32
    range(33, 34): 'Prim. Metals', #33
    range(34, 35): 'Fab. Metals', #34
    range(35, 36): 'Machinery', #35
    range(36, 37): 'Electrical Eq.', #36
    range(37, 38): 'Transport Eq.', #37
    range(38, 40): 'Manufacturing', #38~39
    range(40, 41): 'Railroads', #40
    range(41, 48): 'Other Transport.', #41~47
    range(49, 50): 'Utilities', #49
    range(50, 53): 'Retail',  #50~52
    range(53, 54): 'Dept. Stores', #53
    range(54, 60): 'Retail',  #54~59
    range(60, 70): 'Financial', #60~69
    }

    def map_industry(sic_code):
        sic_prefix = int(str(sic_code)[:2])
        for key in industry_mapping:
            if sic_prefix in key:
                return industry_mapping[key]
        return 'Other'

    df['industry'] = df['SICCD'].apply(map_industry)
    
    # 5. 計算產業 Value Weighted Return
    ind_ret = df.groupby(['year_month', 'industry']) \
            .apply(lambda x: np.average(x['stock_month_ret'], weights = x['monthly_market_value'])) \
            .rename('industry_return').reset_index()

    merged = pd.merge(ind_ret, df, on = ['year_month', 'industry'])
    merged['year_month']+=1
    
    return merged[['PERMNO', 'SICCD', 'year_month', 'industry', 'indmom']].dropna().reset_index(drop = True)

indmom = indmom()
indmom.to_pickle('indmom.pickle')

# mvel1: size
Banz, 1981, JFE

In [11]:
def size():
    # 1. Input Data
    df = data[['date', 'PERMNO', 'PRC', 'SHROUT']].copy()

    df['date'] = pd.to_datetime(df['date'])
    df['year_month'] = df['date'].dt.to_period('M')

    df['PRC'] = pd.to_numeric(np.abs(df['PRC']), errors='coerce')
    df['SHROUT'] = pd.to_numeric(np.abs(df['SHROUT']), errors='coerce')
    df.dropna(subset=['PRC', 'SHROUT'], inplace=True)

    df = df.sort_values(by=['PERMNO', 'year_month'])
    
    # 2. 計算個股市值
    df['market_value'] = df['PRC']*df['SHROUT']
    size = pd.DataFrame(df.groupby(['PERMNO', 'year_month'])['market_value'].agg('mean').rename('stockmv')).reset_index()
    
    # 3. 個股市值/股市總市值
    size['totalmv'] = size.groupby(['year_month'])['stockmv'].transform('sum')
    size['mvel1'] = size['stockmv']/size['totalmv']
    size['year_month']+=1
    # size['mvel1'] = size.groupby('PERMNO')['size%'].shift(1)

    return size[['PERMNO', 'year_month', 'mvel1', 'stockmv']].dropna().reset_index(drop = True)

size = size()
size.to_pickle('mvel1.pickle')